In [ ]:
from hstrader import HsTrader
from hstrader.models import Resolution
import os
from dotenv import load_dotenv
import pandas as pd
import plotly.graph_objects as go
from ipywidgets import widgets, VBox
from IPython.display import display


In [ ]:
# Load environment variables from a .env file
load_dotenv()

# Get the CLIENT_ID from the environment variables
id = os.getenv('CLIENT_ID')

# Get the CLIENT_SECRET from the environment variables
secret = os.getenv('CLIENT_SECRET')

# Initialize the HsTrader client with the client ID and secret
client = HsTrader(id, secret)

### Plot Candlestick Chart with Fibonacci Retracement Indicator (Historical Market Data):

In [ ]:
def get_fibonacci_levels(high, low, uptrend):
    """
    Calculate Fibonacci retracement levels.
    
    Parameters
    ---------
    high (float): The highest price in the range.
    low (float): The lowest price in the range.
    uptrend (bool): True for uptrend, False for downtrend.
    
    Returns:
    dict: A dictionary with Fibonacci retracement levels.
    """
    difference = high - low
    if uptrend:
        return {
            '0.00%': high,
            '23.6%': high - difference * 0.236,
            '38.2%': high - difference * 0.382,
            '50.0%': high - difference * 0.5,
            '61.8%': high - difference * 0.618,
            '100.0%': low
        }
    else:
        return {
            '100.0%': high,
            '61.8%': low + difference * 0.618,
            '50.0%': low + difference * 0.5,
            '38.2%': low + difference * 0.382,
            '23.6%': low + difference * 0.236,
            '0.00%': low
        }

In [ ]:
def plot_fibonacci_retracement(df, fibonacci_df, levels, symbol):
    """
    Plots Fibonacci retracement levels on a candlestick chart.

    Parameters
    -----------
        df (DataFrame): DataFrame containing OHLC price data.
        fibonacci_df (DataFrame): A slice of the original DataFrame.
        levels (dict): Dictionary of Fibonacci retracement levels.
        symbol (str): The symbol name for the plot title.
    """
    fig = go.Figure()

    # Add candlestick chart
    fig.add_trace(go.Candlestick(
        x=df.index,
        open=df['open'],
        high=df['high'],
        low=df['low'],
        close=df['close'],
        name='Candlestick'
    ))

    # Add Fibonacci levels and filled sections
    colors = ['rgba(128, 0, 128, 0.2)', 'rgba(0, 0, 255, 0.2)', 'rgba(0, 128, 0, 0.2)', 'rgba(255, 0, 0, 0.2)', 'rgba(255, 165, 0, 0.2)']
    level_values = list(levels.values())
    labels = list(levels.keys())

    for i in range(len(level_values)):
        # Draw the level lines
        fig.add_trace(go.Scatter(
            x=[fibonacci_df.index.min(), fibonacci_df.index.max()],
            y=[level_values[i], level_values[i]],
            mode='lines',
            line=dict(dash='dash', width=0.7, color=colors[i % len(colors)]),
            name=labels[i],
            showlegend=True,
            legendgroup=labels[i],
            hoverinfo='skip'
        ))

        # Add level labels
        fig.add_trace(go.Scatter(
            x=[fibonacci_df.index.max()],
            y=[level_values[i]],
            text=[labels[i]],
            mode='text',
            showlegend=False,
            legendgroup=labels[i],
            hoverinfo='skip'
        ))

        # Fill the area between levels
        if i > 0:
            fig.add_trace(go.Scatter(
                x=fibonacci_df.index.tolist() + fibonacci_df.index[::-1].tolist(),
                y=[level_values[i-1]] * len(fibonacci_df.index) + [level_values[i]] * len(fibonacci_df.index),
                fill='toself',
                fillcolor=colors[i % len(colors)],
                line=dict(width=0),
                name=f'{labels[i]} Fill',
                showlegend=True,
                hoverinfo='skip'
            ))

    fig.update_layout(
        title=f'Fibonacci Retracement for {symbol.symbol}',
        xaxis_title='Date',
        yaxis_title='Price',
        xaxis_rangeslider_visible=False,
        template='plotly_white',
        legend=dict(
            title='Fibonacci Levels',
            orientation='h',
            yanchor='bottom',
            y=1.1,
            xanchor='right',
            x=1
        ),
        margin=dict(t=150)
    )

    fig.show()


In [ ]:
def fib_retrace(symbol):
    """
    Fetches market data for the given symbol, calculates Fibonacci retracement levels, and plots them.

    Parameters
    ----------
        symbol (str): The symbol symbol for which to fetch market data and plot Fibonacci levels.
    """
    global trend, levels

    
        
    symbol = client.get_symbol(symbol)
    data = client.get_market_history(symbol=symbol.id, resolution=Resolution.H1)
    
    # Create a DataFrame from the retrieved data
    df = pd.DataFrame([bar.model_dump() for bar in data])
    df['time'] = pd.to_datetime(df['time'], utc=True)
    df.set_index('time', inplace=True)
    
    # Display the initial candlestick chart
    fig = go.Figure()
    fig.add_trace(go.Candlestick(
        x=df.index,
        open=df['open'],
        high=df['high'],
        low=df['low'],
        close=df['close'],
        name='Candlestick'
    ))
    fig.show()
    
    trend = None
    levels = None

    # Widgets for date range input and trend selection
    frm_input = widgets.Text(description='From:', placeholder='YYYY-MM-DD HH:MM:SS')
    to_input = widgets.Text(description='To:', placeholder='YYYY-MM-DD HH:MM:SS')
    trend_input = widgets.RadioButtons(
        options=['uptrend', 'downtrend'],
        description='Trend:',
        disabled=False
    )
    button = widgets.Button(description="Show ")

    def update_levels(button):
        global trend, levels
        try:
            frm = pd.to_datetime(frm_input.value, utc=True)
            to = pd.to_datetime(to_input.value, utc=True)
            fibonacci_df = df.loc[frm:to]
            max_value = fibonacci_df['high'].max()
            min_value = fibonacci_df['low'].min()
            trend = True if trend_input.value == 'uptrend' else False
            levels = get_fibonacci_levels(max_value, min_value, trend)
            if levels:
                print("Percentage level\t Price")
                for level, price in levels.items():
                    print(f'{level}\t\t {round(price, 3)}')
                plot_fibonacci_retracement(df, fibonacci_df, levels, symbol)
        except Exception as e:
            print(f"Error: {e}")

    button.on_click(update_levels)
    display(VBox([frm_input, to_input, trend_input, button]))

    

In [ ]:
# Replace with the desired symbol 
fib_retrace('GBPUSD')  

### Example Plot

Below is a static image of Fibonacci Retracement applied to EUR/USD historical data:
![Fibonacci Plot](img/fibonacci.png)

### Plot Real-Time Candlestick Chart with Fibonacci Retracement Indicator :
If you’re interested in backtesting and strategy optimization, the code above utilizes the Fibonacci Retracement Indicator with historical data. However, if you want real-time data for live trading decisions, you can run the code below to incorporate the indicator in live market conditions.

In [ ]:
import os
import logging
import asyncio
import pandas as pd
import plotly.graph_objects as go
from datetime import timedelta
from dotenv import load_dotenv
from hstrader import HsTrader
from hstrader.models import Tick, Event, Resolution
from IPython.display import display, clear_output
import nest_asyncio

In [ ]:
# Enable logging
logging.basicConfig(level=logging.INFO)

# Load environment variables
load_dotenv()

# Get the CLIENT_ID and CLIENT_SECRET from the environment variables
id = os.getenv('CLIENT_ID')
secret = os.getenv('CLIENT_SECRET')

# Initialize the HsTrader client with the client ID and secret
client = HsTrader(id, secret)

In [ ]:
def get_fibonacci_levels(high, low, uptrend):
    """
    Calculate Fibonacci retracement levels.
    
    Parameters
    ---------
    high (float): The highest price in the range.
    low (float): The lowest price in the range.
    uptrend (bool): True for uptrend, False for downtrend.
    
    Returns:
    dict: A dictionary with Fibonacci retracement levels.
    """
    difference = high - low
    if uptrend:
        return {
            '0.00%': high,
            '23.6%': high - difference * 0.236,
            '38.2%': high - difference * 0.382,
            '50.0%': high - difference * 0.5,
            '61.8%': high - difference * 0.618,
            '100.0%': low
        }
    else:
        return {
            '100.0%': high,
            '61.8%': low + difference * 0.618,
            '50.0%': low + difference * 0.5,
            '38.2%': low + difference * 0.382,
            '23.6%': low + difference * 0.236,
            '0.00%': low
        }

In [ ]:
def Plot_Fibonacci_Retracement(df, levels,fig):
    """
    Plots Fibonacci retracement levels on a candlestick chart.

    Parameters
    -----------
        df (DataFrame): DataFrame containing OHLC price data.
        levels (dict): Dictionary of Fibonacci retracement levels.
        fig (plotly.graph_objs.Figure): The Plotly figure to which Fibonacci retracement levels will be added.
    """
    

    # Add Fibonacci levels and filled sections
    colors = ['rgba(128, 0, 128, 0.2)', 'rgba(0, 0, 255, 0.2)', 'rgba(0, 128, 0, 0.2)', 'rgba(255, 0, 0, 0.2)', 'rgba(255, 165, 0, 0.2)']
    level_values = list(levels.values())
    labels = list(levels.keys())

    for i in range(len(level_values)):
        # Draw the level lines
        fig.add_trace(go.Scatter(
            x=[df.index.min(), df.index.max()],
            y=[level_values[i], level_values[i]],
            mode='lines',
            line=dict(dash='dash', width=0.7, color=colors[i % len(colors)]),
            name=labels[i],
            showlegend=True,
            legendgroup=labels[i],
            hoverinfo='skip'
        ))

        # Add level labels
        fig.add_trace(go.Scatter(
            x=[df.index.max()],
            y=[level_values[i]],
            text=[labels[i]],
            mode='text',
            showlegend=False,
            legendgroup=labels[i],
            hoverinfo='skip'
        ))

        # Fill the area between levels
        if i > 0:
            fig.add_trace(go.Scatter(
                x=df.index.tolist() + df.index[::-1].tolist(),
                y=[level_values[i-1]] * len(df.index) + [level_values[i]] * len(df.index),
                fill='toself',
                fillcolor=colors[i % len(colors)],
                line=dict(width=0),
                name=f'{labels[i]} Fill',
                showlegend=True,
                hoverinfo='skip'
            ))

    fig.update_layout(
        title=f'Fibonacci Retracement ',
        xaxis_title='Date',
        yaxis_title='Price',
        xaxis_rangeslider_visible=True,
        template='plotly_white',
        legend=dict(
            title='Fibonacci Levels',
            orientation='h',
            yanchor='bottom',
            y=1.1,
            xanchor='right',
            x=1
        ),
        margin=dict(t=150)
    )

    


In [ ]:
# Retrieve market data for any forex symbol by specifying its name in the get_symbol function
symbol = client.get_symbol('EURUSD')
data = client.get_market_history(symbol=symbol.id, resolution=Resolution.M1)

# Create a DataFrame from the retrieved data
df = pd.DataFrame([bar.model_dump() for bar in data])
df['time'] = pd.to_datetime(df['time'])
df.set_index('time', inplace=True)

# Calculate initial Fibonacci levels
high = df['high'].max()
low = df['low'].min()
trend = True  # Assume an uptrend initially
levels = get_fibonacci_levels(high, low, trend)

In [ ]:
# Setup the initial plot
fig = go.FigureWidget()
candlestick = go.Candlestick(x=df.index, open=df['open'], high=df['high'], low=df['low'], close=df['close'], name='Candlestick')
fig.add_trace(candlestick)
Plot_Fibonacci_Retracement(df,levels,fig)

In [ ]:
# Prepare data dictionary
CANDLE_INTERVAL = timedelta(minutes=1)
data = {
    'x': list(df.index),
    'open': list(df['open']),
    'high': list(df['high']),
    'low': list(df['low']),
    'close': list(df['close'])
}

In [ ]:
@client.subscribe(Event.MARKET)
async def on_market(tick: Tick):
    global data, df, levels

    try:
        if tick.symbol_id == symbol.id:
            tick_time = pd.to_datetime(tick.time)
            if not data['x']:
                data['x'].append(tick_time)
                data['open'].append(tick.bid)
                data['high'].append(tick.bid)
                data['low'].append(tick.bid)
                data['close'].append(tick.bid)
            elif tick_time >= data['x'][-1] + CANDLE_INTERVAL:
                data['x'].append(tick_time)
                data['open'].append(data['close'][-1])
                data['high'].append(tick.bid)
                data['low'].append(tick.bid)
                data['close'].append(tick.bid)
            else:
                data['low'][-1] = min(tick.bid, data['low'][-1])
                data['high'][-1] = max(tick.bid, data['high'][-1])
                data['close'][-1] = tick.bid

            df = pd.DataFrame({
                'time': data['x'],
                'open': data['open'],
                'high': data['high'],
                'low': data['low'],
                'close': data['close']
            }).set_index('time')

            # Recalculate the Fibonacci levels
            high = df['high'].max()
            low = df['low'].min()
            levels = get_fibonacci_levels(high, low, trend)
    except Exception as e:
        logging.error(f"Error in on_market: {e}")
        await client.refresh_token()

In [ ]:
async def update_plot():
    global data, df, levels
    while True:
        await asyncio.sleep(1)
        clear_output(wait=True)
        new_fig = go.Figure()
        new_fig.add_trace(go.Candlestick(x=df.index, open=df['open'], high=df['high'], low=df['low'], close=df['close'], name='Candlestick'))
        Plot_Fibonacci_Retracement(df,levels,new_fig)
        

        display(new_fig)

In [ ]:
nest_asyncio.apply()
loop=asyncio.get_event_loop()
loop.run_until_complete(asyncio.gather(client.start_async(), update_plot()))